# ADD ELEMENTS TO THE ONTOLOGY OF EUCLID'S BOOK 1

In [ ]:
from __future__ import annotations

import pathlib
import sys

NOTEBOOK_DIR = pathlib.Path.cwd()
if '_NB_SYS_PATH_ADJUSTED' not in globals():
    sys.path.insert(0, str(NOTEBOOK_DIR))
    _NB_SYS_PATH_ADJUSTED = True

import pandas as pd
import rdflib

from modules import legacy_utils
from modules import rdf_utils

URIREF = rdflib.URIRef
RDF_TYPE = rdflib.RDF.type
RDFS_LABEL = rdflib.RDFS.label
SKOS_PREFLABEL = rdflib.SKOS.prefLabel
CONCEPT_CLASS = URIREF("https://www.foom.com/core#concept")
CONTAINS_CONCEPT = URIREF("https://www.foom.com/core#contains_concept")
IS_CONCEPT_IN = URIREF("https://www.foom.com/core#is_concept_in")
NAMESPACE = "<https://www.foom.com/core>"

## CONCEPTUAL COMPONENTS OF POSTULATES AND COMMON NOTIONS

In [ ]:
# add contains_concepts relations to postulates and common notions
from modules import legacy_utils
from modules import file_utils
from modules import rdf_utils

INPUT_FILE = NOTEBOOK_DIR / "input" / "addenda_contains_concept.csv"
OUTPUT_FOLDER = NOTEBOOK_DIR / "ontologies"


def add_concept(
    g: rdflib.Graph,
    concept_string: str
):
    concept_label = concept_string.replace("_", " ")
    concept_iri = legacy_utils.create_iri(f"Concept: {concept_string}")
    g.add((concept_iri, RDF_TYPE, CONCEPT_CLASS))
    g.add((concept_iri, RDFS_LABEL, rdflib.Literal(f"Concept: {concept_label}")))
    g.add((concept_iri, SKOS_PREFLABEL, rdflib.Literal(concept_label)))
    return g, concept_iri


def contains_concepts(
    g: rdflib.Graph,
    df: pd.DataFrame = INPUT_FILE
):
    concepts = set()
    for _, row in df.iterrows():
        subject_iri = URIREF(row['subject'])
        objects = {concept.strip() for concept in row['objects'].split(',')}
        for object_string in objects:
            object_iri = legacy_utils.create_iri(f"Concept: {object_string}")
            concepts.add(object_string)
            # add triples to the graph
            g.add((
                subject_iri,
                CONTAINS_CONCEPT,
                object_iri))
            g.add((
                object_iri,
                IS_CONCEPT_IN,
                subject_iri))
    
    print(f"Added {len(concepts)} new concepts to the graph.")
    return g, concepts


def add_concepts(
    g: rdflib.Graph, 
    concepts: set[str]
):
    iris = set()
    for concept_string in concepts:
        g, concept_iri = add_concept(g, concept_string)
        iris.add(concept_iri)
    iris_df = pd.DataFrame({"iri": sorted(iris)})
    return g, iris_df
    
def add_concepts_and_contains_concept_from_postulates_and_common_notions(
    g: rdflib.Graph,
    input_file: pathlib.Path = INPUT_FILE
):
    # read csv file
    df = pd.read_csv(input_file)

    # add contains_concept relations
    g, concepts = contains_concepts(g, df)

    # add concept under the Concept class
    g, iris_df = add_concepts(g, concepts, df)

    # output graph to ttl file
    rdf_utils.save_graph_with_timestamp(
        g, 
        output_subdir=OUTPUT_FOLDER)

    # output csv
    file_utils.write_csv(
        iris_df, 
        filename="new_concepts_from_postulates_common_notions.csv", 
        output_dir="output")
    return g

#  read input ontology file
INPUT_TTL = file_utils.latest_ontology_file()
g = rdf_utils.load_graph(INPUT_TTL)

# add concepts and contains_concept relations
g = add_concepts_and_contains_concept_from_postulates_and_common_notions(g, INPUT_FILE)

## CONCEPTUAL COMPONENTS OF ALL CONCEPTS

In [ ]:
from modules import rdf_utils
INPUT_CSV = NOTEBOOK_DIR / "input" / "conceptual_components.csv"
OUTPUT_FOLDER = NOTEBOOK_DIR / "ontologies"

def add_conceptual_components(
    g: rdflib.Graph,
    input_csv: pathlib.Path = INPUT_CSV
):
    # read csv file with concepts and their conceptual components
    df = pd.read_csv(input_csv).fillna("")

    # iterate over rows and 
    # add triples <s, oai:contains_concept, o> and
    # <o, oai:is_concept_in, s>
    # to the graph
    for _, row in df.iterrows():
        subject_iri = URIREF(row['concept_iri'])
        if component_values := row['component']:
            components = {component.strip() for component in component_values.split(';')}
            for component_string in components:
                component_iri = URIREF(component_string)
                # add triples to the graph
                g.add((
                    subject_iri,
                    CONTAINS_CONCEPT,
                    component_iri))
                g.add((
                    component_iri,
                    IS_CONCEPT_IN,
                    subject_iri))
    
    print("Length of graph after adding conceptual components:", len(g))
    rdf_utils.save_graph_with_timestamp(
        g, 
        output_subdir=OUTPUT_FOLDER,)
    return g

#  read input ontology file
INPUT_TTL = file_utils.latest_ontology_file()
g = rdf_utils.load_graph(INPUT_TTL)

# add conceptual components to concepts
g = add_conceptual_components(g, INPUT_CSV)